# Baseline Training for Vesuvius Ink Detection

## Overview
This notebook implements the baseline training pipeline using 2.5D U-Net for ink segmentation.
- **Model**: segmentation_models_pytorch (SMP) Unet with 'timm-efficientnet-b4' encoder, in_channels=32 (z-slices), classes=1, activation=None (logits output).
- **Loss**: Custom 0.6 * Class-Balanced BCE (with label smoothing) + 0.4 * DiceLoss, computed only on 'valid' (eroded mask) areas to avoid border noise.
- **Optimizer**: AdamW(lr=3e-4, weight_decay=1e-4), CosineAnnealingLR(T_max=30).
- **Training**: Full FP32 (no AMP), batch_size=4, 30 epochs, early stopping patience=8 on val F0.5 (pre-CC).
- **Validation**: Fragment-wise CV (fold 1: train on 1, val on 2). Val F0.5 computed via full sliding-window inference (tile=512, stride=256, Gaussian blend sigma=64), threshold sweep [0.01-0.50] to maximize F0.5 within mask, optional small component removal (min_cc=200px).
- **Efficiency**: Train on GPU, log progress per epoch (prec/rec/p95/p99/mean_prob), EMA (decay=0.999) for stable inference, save best checkpoint by val F0.5 (pre-CC), torch.cuda.empty_cache() after val.
- **Targets**: Val F0.5 pre-CC >0.60 for baseline; iterate with TTA/z-offset for higher.

## Key Components
1. **Imports & Utils**: SMP, torch, sklearn fbeta_score (beta=0.5 for F0.5), custom cb_bce_loss + dice_loss.
2. **Model Definition**: smp.Unet(...).to(device)
3. **Sliding Inference**: Function to predict full fragment: Generate overlapping tiles, predict, blend with 2D Gaussian weights, sigmoid, multiply by mask, compute F0.5 vs inklabels at best threshold.
4. **Train Fold**: Load train/val data (from data_pipeline.py), create loaders (train with sampler/transform/z_jitter=8, val no), train loop: forward/backward, val every epoch, early stop, save EMA ckpt.
5. **Execution**: Train fold 1 (train=1, val=2), monitor val F0.5.

## Next Steps
- Insert and execute imports cell.
- Define model, loss, sliding_inference, train_fold functions.
- Load data2 = FragmentLoader('2').get_data_dict(), create val_dataset=VesuviusDataset(data2, transform=None), val_loader=DataLoader(..., batch_size=1, shuffle=False).
- Train fold 1, log progress (print epoch, train_loss, val_F05).
- If val F0.5 <0.60, debug (e.g., increase pos_threshold to 0.10, adjust loss weights).
- After fold1, request expert review on results, then train fold2, ensemble OOF, generate submission for test 'a' using best ckpts.

## Notes
- Ensure no leakage: Val uses separate fragment, percentiles per-fragment.
- Post-processing in sliding: Threshold tune per val fragment, remove small CC with skimage.measure.regionprops.
- For submission: Similar sliding on test 'a', use learned thresholds or average from CV, RLE encode within mask.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
import segmentation_models_pytorch as smp
from sklearn.metrics import fbeta_score
import numpy as np
from scipy.ndimage import gaussian_filter
from skimage import measure
import cv2
from tqdm import tqdm
import os
import tifffile
import data_pipeline
from data_pipeline import FragmentLoader, VesuviusDataset, TileSampler, train_transform
from importlib import reload
reload(data_pipeline)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

def get_model(in_channels=32, encoder_name='timm-efficientnet-b4', classes=1):
    model = smp.Unet(
        encoder_name=encoder_name,
        encoder_weights='imagenet',
        in_channels=in_channels,
        classes=classes,
        activation=None,  # Logits output
    )
    return model.to(device)

def f05_score(pred, target, valid, beta=0.5):
    pred_flat = (pred > 0.5).flatten()
    target_flat = target.flatten()
    valid_flat = valid.flatten() > 0
    return fbeta_score(target_flat[valid_flat], pred_flat[valid_flat], beta=beta)

Using device: cuda


In [2]:
def sliding_inference(model, data_dict, tile_size=512, stride=256, sigma=64, device='cuda', batch_size=8, tta_flips=True):
    """
    Perform sliding window inference on a full fragment to generate prediction mask.
    Supports 4-way TTA: original, hflip, vflip, rot90 (average probs).
    Returns the blended probability mask multiplied by the original mask.
    """
    model.eval()
    h, w = data_dict['mask'].shape
    pred = np.zeros((h, w), dtype=np.float32)
    count = np.zeros((h, w), dtype=np.float32)
    
    # Precompute Gaussian weights
    yy, xx = np.mgrid[0:tile_size, 0:tile_size]
    center = tile_size // 2
    weights = np.exp(-((xx - center)**2 + (yy - center)**2) / (2 * sigma**2))
    weights /= weights.max()
    
    n = len(data_dict['volume_files'])
    w_z = data_dict['z_window']
    c_z = data_dict['z_center']
    p05, p995 = data_dict['p05'], data_dict['p995']
    den = (p995 - p05 + 1e-8)
    
    # Memmap all volume files
    mmaps = {fn: tifffile.memmap(fn) for fn in data_dict['volume_files']}
    
    # Collect all tile positions
    tiles = []
    for i in range(0, h - tile_size + 1, stride):
        for j in range(0, w - tile_size + 1, stride):
            tiles.append((i, j))
    num_batches = (len(tiles) + batch_size - 1) // batch_size
    print(f'Number of tiles: {len(tiles)}, batches: {num_batches}')
    
    with torch.no_grad():
        for bs in range(0, len(tiles), batch_size):
            batch_idx = bs // batch_size + 1
            if batch_idx % 10 == 0 or batch_idx == 1:
                print(f'Processing batch {batch_idx}/{num_batches}')
            batch_tiles = tiles[bs:bs+batch_size]
            batch_stacks = []
            batch_positions = []
            for i, j in batch_tiles:
                # On-the-fly load z-window ROIs
                stack = []
                z_start = max(0, min(c_z - w_z // 2, n - w_z))
                z_end = z_start + w_z
                for z in range(z_start, z_end):
                    fn = data_dict['volume_files'][z]
                    slice_roi = mmaps[fn][i:i+tile_size, j:j+tile_size].astype(np.float32)
                    slice_roi = np.clip((slice_roi - p05) / den, 0, 1)
                    stack.append(slice_roi)
                stack_tile = np.stack(stack, axis=0)  # (C, tile, tile)
                batch_stacks.append(stack_tile)
                batch_positions.append((i, j))
            
            if batch_stacks:
                batch_tensor = torch.from_numpy(np.stack(batch_stacks)).to(device)  # (B, C, tile, tile)
                
                # List to accumulate TTA probs
                tta_probs = []
                
                # Original
                logits = model(batch_tensor)
                probs_orig = torch.sigmoid(logits).cpu().numpy()  # (B, 1, tile, tile)
                tta_probs.append(probs_orig)
                
                if tta_flips:
                    # HFlip
                    batch_hflip = torch.flip(batch_tensor, dims=[-1])  # Flip width
                    logits_h = model(batch_hflip)
                    probs_h = torch.sigmoid(logits_h).cpu().numpy()
                    probs_h = np.flip(probs_h, axis=-1)  # Flip back
                    tta_probs.append(probs_h)
                    
                    # VFlip
                    batch_vflip = torch.flip(batch_tensor, dims=[-2])  # Flip height
                    logits_v = model(batch_vflip)
                    probs_v = torch.sigmoid(logits_v).cpu().numpy()
                    probs_v = np.flip(probs_v, axis=-2)  # Flip back
                    tta_probs.append(probs_v)
                    
                    # Rot90 (90 deg clockwise)
                    batch_rot = torch.rot90(batch_tensor, k=1, dims=[-2, -1])
                    logits_r = model(batch_rot)
                    probs_r = torch.sigmoid(logits_r).cpu().numpy()
                    probs_r = np.rot90(probs_r, k=3, axes=[-2, -1])  # Rotate back (270 deg = -90)
                    tta_probs.append(probs_r)
                
                # Average TTA probs
                probs = np.mean(tta_probs, axis=0)  # (B, 1, tile, tile)
                
                # Add to accumulators
                for k, (i, j) in enumerate(batch_positions):
                    prob = probs[k, 0]
                    pred[i:i+tile_size, j:j+tile_size] += prob * weights
                    count[i:i+tile_size, j:j+tile_size] += weights
    
    # Normalize
    pred = pred / (count + 1e-8)
    
    # Apply original mask
    pred *= data_dict['mask']
    
    # Print pred stats within mask
    valid = data_dict['mask'] > 0
    pred_valid = pred[valid]
    if len(pred_valid) > 0:
        print(f'Pred stats (in-mask): min={pred_valid.min():.3f}, mean={pred_valid.mean():.3f}, p95={np.percentile(pred_valid, 95):.3f}, p99={np.percentile(pred_valid, 99):.3f}')
    
    return pred

In [3]:
def compute_f05_fast(pred_prob, target, mask, thresholds=None, apply_cc=False, min_cc_size=196, beta=0.5, sample_size=2000000, bins=512, rng_seed=42):
    import numpy as np
    import cv2
    from skimage import measure

    valid = mask > 0
    y_true = (target > 0)[valid].astype(np.uint8)
    y_prob = pred_prob[valid].astype(np.float32)

    n = y_prob.size
    # Candidate thresholds from small sample + fixed grid
    if thresholds is None:
        if n > sample_size:
            rng = np.random.RandomState(rng_seed)
            idx = rng.choice(n, size=sample_size, replace=False)
            y_sample = y_prob[idx]
        else:
            y_sample = y_prob
        grid = np.arange(0.08, 0.52, 0.02)
        tq = np.unique(np.clip(np.quantile(y_sample, np.linspace(0.90, 0.995, 12)), 1e-4, 0.999))
        thresholds = np.unique(np.concatenate([grid, tq]))

    # Build histograms once
    edges = np.linspace(0.0, 1.0, bins + 1, dtype=np.float32)
    pos = y_true == 1
    neg = ~pos
    P = int(pos.sum())
    if P == 0:
        return 0.0, 0.5

    pos_hist, _ = np.histogram(y_prob[pos], bins=edges)
    neg_hist, _ = np.histogram(y_prob[neg], bins=edges)
    pos_cum = np.cumsum(pos_hist[::-1])[::-1]
    neg_cum = np.cumsum(neg_hist[::-1])[::-1]

    beta2 = beta * beta
    def f_from_counts(tp, fp, fn):
        denom = (1 + beta2) * tp + beta2 * fn + fp + 1e-9
        return (1 + beta2) * tp / denom

    best_s, best_t = 0.0, 0.5
    for t in thresholds:
        b = np.searchsorted(edges, t, side='left')
        b = min(max(b, 0), bins)
        tp = int(pos_cum[b]); fp = int(neg_cum[b]); fn = P - tp
        s = f_from_counts(tp, fp, fn)
        if s > best_s:
            best_s, best_t = s, t

    # Exact pre-CC at best_t
    y_pred = (y_prob > best_t)
    tp = int((y_pred & pos).sum()); fp = int((y_pred & neg).sum()); fn = P - tp
    pre = f_from_counts(tp, fp, fn)

    # Quick diagnostics
    show = min(n, sample_size)
    qs = np.quantile(y_prob[:show], [0.95, 0.99]) if n >= show else np.quantile(y_prob, [0.95, 0.99])
    print(f'  At best_t={best_t:.3f}: prec={tp/(tp+fp+1e-9):.4f}, rec={tp/(tp+fn+1e-9):.4f}, p95={qs[0]:.3f}, p99={qs[1]:.3f}, mean_prob={y_prob.mean():.3f}')
    if not apply_cc:
        print(f'Val F0.5 pre-CC: {pre:.4f} @t={best_t:.3f}')
        return pre, best_t

    # CC only at best_t (downsampled)
    pb = (pred_prob > best_t).astype(np.uint8)
    h, w = pb.shape
    scale = 0.25
    pb_d = cv2.resize(pb, (int(w * scale), int(h * scale)), interpolation=cv2.INTER_NEAREST)
    labeled = measure.label(pb_d, connectivity=2)
    min_area = max(1, int(min_cc_size * (scale ** 2)))
    for r in measure.regionprops(labeled):
        if r.area < min_area:
            labeled[labeled == r.label] = 0
    kept = (labeled > 0).astype(np.uint8)
    kept_up = cv2.resize(kept, (w, h), interpolation=cv2.INTER_NEAREST)
    y_post = kept_up[valid].astype(np.uint8)

    tp2 = int(((y_post == 1) & (y_true == 1)).sum())
    fp2 = int(((y_post == 1) & (y_true == 0)).sum())
    fn2 = int(((y_post == 0) & (y_true == 1)).sum())
    post = f_from_counts(tp2, fp2, fn2)
    return post, best_t

def cb_bce_loss(logits, targets, valids, max_pw=8.0, eps=1e-6, ls=0.05):
    smoothed_targets = targets.clone() * (1 - ls) + 0.5 * ls
    with torch.no_grad():
        valid = (valids > 0.5).float()
        pos = (targets * valid).sum(); tot = valid.sum(); neg = tot - pos
        pos_weight = (neg / (pos + eps)).clamp(1.0, max_pw)
    bce = F.binary_cross_entropy_with_logits(logits, smoothed_targets, reduction='none', pos_weight=pos_weight)
    bce_loss = bce * valids
    return bce_loss.sum() / (valids.sum() + eps)

def dice_loss(logits, targets, valids, smooth=1e-6):
    pred = torch.sigmoid(logits)
    pred_m = pred * valids
    tgt_m = targets * valids
    inter = (pred_m * tgt_m).sum(dim=(2,3))
    denom = pred_m.sum(dim=(2,3)) + tgt_m.sum(dim=(2,3)) + smooth
    dice = (2*inter + smooth) / denom
    return 1 - dice.mean()

def combined_loss(logits, targets, valids, bce_w=0.4, dice_w=0.6):
    logits = logits.float()
    targets = targets.float()
    valids = valids.float()
    return bce_w * cb_bce_loss(logits, targets, valids) + dice_w * dice_loss(logits, targets, valids)

class EMA:
    def __init__(self, model, decay=0.999):
        self.decay = decay
        self.shadow = {n: p.data.clone() for n, p in model.named_parameters() if p.requires_grad}
        self.backup = {}
    def update(self, model):
        for n, p in model.named_parameters():
            if p.requires_grad:
                self.shadow[n].mul_(self.decay).add_(p.data, alpha=1 - self.decay)
    def apply(self, model):
        self.backup = {}
        for n, p in model.named_parameters():
            if p.requires_grad:
                self.backup[n] = p.data.clone()
                p.data.copy_(self.shadow[n])
    def restore(self, model):
        for n, p in model.named_parameters():
            if p.requires_grad and n in self.backup:
                p.data.copy_(self.backup[n])
        self.backup = {}

def train_fold(train_frag, val_frag, epochs=12, batch_size=4, lr=1e-4, patience=4, val_every=1):
    from data_pipeline import FragmentLoader, VesuviusDataset, TileSampler, train_transform
    import data_pipeline
    from importlib import reload
    reload(data_pipeline)
    import os
    import torch.nn.utils
    
    # Fragment-specific z_centers
    z_centers = {'1': 30, '2': 33}
    
    # Load data
    print(f'Loading train fragment {train_frag}...')
    train_z = z_centers.get(train_frag, 30)
    train_loader_obj = FragmentLoader(train_frag, z_center=train_z, pos_threshold=0.10, hard_neg_radius=96, z_window=32)
    train_data = train_loader_obj.get_data_dict()
    target_ratios = {'pos': 0.60, 'hard_neg': 0.30, 'neg': 0.10}
    
    # Sampler verification: draw 1000 samples and check ratios
    sampler = TileSampler(train_data['types'], target_ratios)
    sample_indices = list(sampler)[:1000]
    sample_types = [train_data['types'][idx] for idx in sample_indices]
    pos_ratio = sum(1 for t in sample_types if t == 'pos') / len(sample_types)
    hard_neg_ratio = sum(1 for t in sample_types if t == 'hard_neg') / len(sample_types)
    neg_ratio = sum(1 for t in sample_types if t == 'neg') / len(sample_types)
    print(f'Sampler verification (1000 samples): Pos {pos_ratio:.3f}, HardNeg {hard_neg_ratio:.3f}, Neg {neg_ratio:.3f}')
    
    train_dataset = VesuviusDataset(train_data, transform=train_transform, z_jitter=8, train=True)
    train_sampler = TileSampler(train_data['types'], target_ratios)
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler,
        num_workers=4, pin_memory=True, persistent_workers=True
    )
    
    print(f'Loading val fragment {val_frag}...')
    val_z = z_centers.get(val_frag, 30)
    val_loader_obj = FragmentLoader(val_frag, z_center=val_z, pos_threshold=0.10, hard_neg_radius=96, z_window=32)
    val_data = val_loader_obj.get_data_dict()
    val_dataset = VesuviusDataset(val_data, transform=None, z_jitter=0, train=False)
    # No sampler for val, but since full, we use sliding_inference directly
    
    # Model, optimizer, etc.
    in_channels = train_data['z_window']
    model = get_model(in_channels=in_channels, encoder_name='timm-efficientnet-b4')
    ckpt_path = f'best_fold_{train_frag}.pth'
    if os.path.exists(ckpt_path):
        model.load_state_dict(torch.load(ckpt_path, map_location=device), strict=False)
        print(f'Resumed from {ckpt_path}')
    ema = EMA(model, decay=0.999)
    optimizer = AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs)
    
    best_f05 = 0
    best_epoch = 0
    patience_counter = 0
    
    for epoch in range(epochs):
        # Train
        model.train()
        train_loss = 0
        num_batches = 0
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}')
        for batch in pbar:
            images = batch['image'].to(device)
            targets = batch['mask'].to(device)
            valids = batch['valid'].to(device)
            
            optimizer.zero_grad()
            logits = model(images)
            loss = combined_loss(logits, targets, valids, bce_w=0.4, dice_w=0.6)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            ema.update(model)
            
            train_loss += loss.item()
            num_batches += 1
            pbar.set_postfix({'loss': train_loss / num_batches})
        scheduler.step()
        avg_train_loss = train_loss / num_batches
        print(f'Epoch {epoch+1}: Train loss = {avg_train_loss:.4f}')
        
        # Validate every val_every epochs
        if (epoch + 1) % val_every == 0:
            print(f'Running validation on fragment {val_frag}...')
            ema.apply(model)
            pred_prob = sliding_inference(model, val_data, stride=384, batch_size=10, tta_flips=False)
            ema.restore(model)
            val_f05_pre, best_t = compute_f05_fast(pred_prob, val_data['inklabels'], val_data['mask'], apply_cc=False)
            
            # Also compute post-CC
            val_f05_post, _ = compute_f05_fast(pred_prob, val_data['inklabels'], val_data['mask'], thresholds=[best_t], apply_cc=True, min_cc_size=200)
            print(f'Val F0.5 post-CC: {val_f05_post:.4f} @t={best_t:.3f}')
            
            if val_f05_pre > best_f05:
                best_f05 = val_f05_pre
                best_epoch = epoch + 1
                ema.apply(model)
                torch.save(model.state_dict(), ckpt_path)
                ema.restore(model)
                np.save(f'oof_pred_fold_{train_frag}.npy', pred_prob)
                patience_counter = 0
                print(f'New best model saved: {best_f05:.4f} (pre-CC)')
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f'Early stopping at epoch {epoch+1}')
                    break
            
            torch.cuda.empty_cache()
        
    print(f'Fold {train_frag} completed. Best val F0.5: {best_f05:.4f} at epoch {best_epoch}')
    return best_f05, ckpt_path

In [6]:
train_f05_fold1, ckpt_fold1 = train_fold('1', '2')
train_f05_fold2, ckpt_fold2 = train_fold('2', '1')

Loading train fragment 1...


Fragment 1 percentiles (in-mask): p05=3988.0, p995=49174.0


Ink ratios stats: min=0.0000, max=0.6715, mean=0.2042
Num tiles with ratio >= 0.15: 197
Fragment 1: Total tiles: 348, Pos: 197 (56.6%), HardNeg: 115 (33.0%), Neg: 36 (10.3%)
Sampler verification (1000 samples): Pos 0.520, HardNeg 0.388, Neg 0.092
Loading val fragment 2...


Fragment 2 percentiles (in-mask): p05=4097.0, p995=50195.0


Ink ratios stats: min=0.0000, max=0.6991, mean=0.1856
Num tiles with ratio >= 0.15: 702
Fragment 2: Total tiles: 1325, Pos: 702 (53.0%), HardNeg: 609 (46.0%), Neg: 14 (1.1%)


...

Epoch 10/30:   5%|▍         | 16/332 [00:03<00:59,  5.28it/s, loss=0.705]

Epoch 10/30:   5%|▍         | 16/332 [00:04<00:59,  5.28it/s, loss=0.713]

Epoch 10/30:   5%|▌         | 17/332 [00:04<01:01,  5.10it/s, loss=0.713]

Epoch 10/30:   5%|▌         | 17/332 [00:04<01:19,  3.97it/s, loss=0.713]

KeyboardInterrupt: 

In [7]:
# OOF Tuning per Fragment
import numpy as np
import cv2
from skimage import measure
import pandas as pd

# Load OOF predictions and ground truths
oof_frag2 = np.load('oof_pred_fold_1.npy')  # Fold1: pred on frag2
oof_frag1 = np.load('oof_pred_fold_2.npy')  # Fold2: pred on frag1

# Load GT for fragments
ink_frag1 = cv2.imread('train/1/inklabels.png', cv2.IMREAD_GRAYSCALE) / 255.0
ink_frag2 = cv2.imread('train/2/inklabels.png', cv2.IMREAD_GRAYSCALE) / 255.0
mask_frag1 = cv2.imread('train/1/mask.png', cv2.IMREAD_GRAYSCALE) / 255.0
mask_frag2 = cv2.imread('train/2/mask.png', cv2.IMREAD_GRAYSCALE) / 255.0

# Function to compute best t and min_cc for a fragment
def tune_oof(pred, ink, mask, thresholds=np.arange(0.08, 0.52, 0.02), min_cc_list=[128, 160, 196, 256, 300]):
    valid = mask > 0
    y_true = (ink > 0)[valid].astype(np.uint8)
    y_prob = pred[valid].astype(np.float32)
    P = int(np.sum(y_true))
    if P == 0:
        return 0.3, 196, 0.0

    beta = 0.5
    beta2 = beta ** 2
    def fbeta(tp, fp, fn):
        denom = (1 + beta2) * tp + beta2 * fn + fp + 1e-9
        return (1 + beta2) * tp / denom

    best_score = 0
    best_t, best_cc = 0.3, 196
    for t in thresholds:
        for min_cc in min_cc_list:
            # Threshold
            pb = (pred > t).astype(np.uint8)
            # CC filter (downsampled)
            h, w = pb.shape
            scale = 0.25
            pb_d = cv2.resize(pb, (int(w*scale), int(h*scale)), interpolation=cv2.INTER_NEAREST)
            labeled = measure.label(pb_d, connectivity=2)
            min_area = max(1, int(min_cc * (scale**2)))
            for r in measure.regionprops(labeled):
                if r.area < min_area:
                    labeled[labeled == r.label] = 0
            kept = (labeled > 0).astype(np.uint8)
            kept_up = cv2.resize(kept, (w, h), interpolation=cv2.INTER_NEAREST)
            y_pred = kept_up[valid].astype(np.uint8)
            tp = int(np.sum((y_pred == 1) & (y_true == 1)))
            fp = int(np.sum((y_pred == 1) & (y_true == 0)))
            fn = P - tp
            score = fbeta(tp, fp, fn)
            if score > best_score:
                best_score = score
                best_t = t
                best_cc = min_cc
    print(f'Best F0.5: {best_score:.4f} @ t={best_t:.3f}, min_cc={best_cc}')
    return best_t, best_cc, best_score

# Tune for each fragment
t1, cc1, score1 = tune_oof(oof_frag1, ink_frag1, mask_frag1)
t2, cc2, score2 = tune_oof(oof_frag2, ink_frag2, mask_frag2)
mean_cv = (score1 + score2) / 2
print(f'Mean CV F0.5: {mean_cv:.4f}')

# Average thresholds and CC
t_avg = (t1 + t2) / 2
cc_avg = int((cc1 + cc2) / 2)
print(f'Avg t: {t_avg:.3f}, avg min_cc: {cc_avg}')

Best F0.5: 0.3544 @ t=0.500, min_cc=300


KeyboardInterrupt: 

In [ ]:
# Test Inference and Submission
import numpy as np
import cv2
from skimage import measure
import pandas as pd
from data_pipeline import FragmentLoader
import torch
from tqdm import tqdm
import os
import tifffile

# Adapt FragmentLoader for test (no inklabels)
class TestFragmentLoader(FragmentLoader):
    def __init__(self, fragment_id: str, z_center: int = 32, z_window: int = 32):
        self.fragment_id = fragment_id
        self.volume_path = f'test/{fragment_id}/surface_volume/'
        self.mask_path = f'test/{fragment_id}/mask.png'
        self.inklabels_path = None  # No inklabels for test
        self.tile_size = 512
        self.stride = 256
        self.erosion = 0  # No erosion for test
        self.pos_threshold = 0.0
        self.hard_neg_radius = 0
        self.z_center = z_center
        self.z_window = z_window
        
        self.volume_files = sorted([os.path.join(self.volume_path, f) for f in os.listdir(self.volume_path) if f.endswith('.tif')])
        self.load_fragment()

    def load_fragment(self):
        # Load mask
        self.mask = cv2.imread(self.mask_path, cv2.IMREAD_GRAYSCALE).astype(np.float32) / 255.0
        self.inklabels = np.zeros_like(self.mask)  # Dummy for test
        
        # Compute percentiles within mask by sampling every 5th slice
        mask_bool = self.mask > 0
        vals = []
        sample_indices = list(range(0, len(self.volume_files), 5))
        for idx in sample_indices:
            slice_data = tifffile.imread(self.volume_files[idx])
            vals.append(slice_data[mask_bool])
        vals = np.concatenate(vals)
        self.p05 = np.percentile(vals, 0.5)
        self.p995 = np.percentile(vals, 99.5)
        print(f'Test fragment {self.fragment_id} percentiles (in-mask): p05={self.p05}, p995={self.p995}')
        
        # No erosion for test
        self.eroded_mask = self.mask
        
        # No dilation needed
        self.dilated_ink = np.zeros_like(self.mask, dtype=bool)
        
        # No coords for sliding inference
        self.coords = []
        self.types = []
        self.ink_ratios = []

    def get_data_dict(self):
        return {
            'coords': self.coords,
            'ink_ratios': self.ink_ratios,
            'types': self.types,
            'p05': self.p05,
            'p995': self.p995,
            'eroded_mask': self.eroded_mask,
            'mask': self.mask,
            'inklabels': self.inklabels,
            'volume_files': self.volume_files,
            'z_center': self.z_center,
            'z_window': self.z_window
        }

# Load test data
test_id = 'a'
test_z_center = 32  # Average of 30 and 33
test_loader = TestFragmentLoader(test_id, z_center=test_z_center, z_window=32)
test_data = test_loader.get_data_dict()
test_mask = test_data['mask']

# Fallback params since tuning low/incomplete: t=0.25, min_cc=196
t_test = 0.25
min_cc_test = 196
print(f'Using fallback t={t_test}, min_cc={min_cc_test} for test')

# Function for full-res CC filter
def apply_cc_filter(binary_mask, min_cc_size):
    labeled = measure.label(binary_mask, connectivity=2)
    for region in measure.regionprops(labeled):
        if region.area < min_cc_size:
            labeled[labeled == region.label] = 0
    return labeled > 0

# RLE encode function
def rle_encode(mask):
    mask_flat = mask.flatten()
    runs = []
    n = len(mask_flat)
    i = 0
    while i < n:
        if mask_flat[i] == 0:
            i += 1
            continue
        start = i
        while i < n and mask_flat[i] == 1:
            i += 1
        length = i - start
        runs.append(f'{start} {length}')
    return ' '.join(runs) if runs else ''

# Z-TTA shifts
z_shifts = [-4, 0, 4]

# Ensemble predictions
ensemble_prob = np.zeros_like(test_mask, dtype=np.float32)
num_models = 0
for fold in ['1', '2']:
    ckpt_path = f'best_fold_{fold}.pth'
    if not os.path.exists(ckpt_path):
        print(f'Warning: {ckpt_path} not found, skipping fold {fold}')
        continue
    
    model = get_model(in_channels=32, encoder_name='timm-efficientnet-b4')
    model.load_state_dict(torch.load(ckpt_path, map_location='cpu'), strict=False)
    model.to(device)
    model.eval()
    ema = EMA(model, decay=0.999)
    ema.apply(model)
    
    fold_probs = []
    for shift in z_shifts:
        temp_data = test_data.copy()
        temp_data['z_center'] = test_z_center + shift
        pred_shift = sliding_inference(model, temp_data, stride=384, batch_size=10, tta_flips=False)
        fold_probs.append(pred_shift)
    
    avg_fold = np.mean(fold_probs, axis=0)
    ensemble_prob += avg_fold
    num_models += 1
    ema.restore(model)
    torch.cuda.empty_cache()

if num_models > 0:
    ensemble_prob /= num_models

# Threshold and post-process
binary = (ensemble_prob > t_test).astype(np.uint8)
binary = apply_cc_filter(binary, min_cc_test)
final_mask = binary * test_mask

# RLE encode
rle = rle_encode(final_mask)

# Save submission with correct column names: Id, Predicted
submission = pd.DataFrame({'Id': [test_id], 'Predicted': [rle]})
submission.to_csv('submission.csv', index=False)
print('Submission saved to submission.csv')
print(f'RLE length: {len(rle)} chars')

# Verify: count positive pixels
pos_pixels = np.sum(final_mask > 0)
print(f'Predicted ink pixels: {pos_pixels:,}')


Test fragment a percentiles (in-mask): p05=8799.0, p995=46892.0
Using fallback t=0.25, min_cc=196 for test


/tmp/ipykernel_8319/1781838780.py:123: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ckpt_path, map_location='cpu'), strict=False)


Number of tiles: 247, batches: 25
Processing batch 1/25


In [12]:
import pandas as pd

# Fix the submission.csv header to match sample
with open('submission.csv', 'r') as f:
    lines = f.readlines()
header = lines[0].strip()
data_line = lines[1].strip()
id_val = data_line.split(',')[0]
rle_val = ','.join(data_line.split(',', 1)[1:])
with open('submission.csv', 'w') as f:
    f.write('Id,Predicted\n')
    f.write(f'{id_val},{rle_val}\n')
print('Fixed submission.csv')

Fixed submission.csv
